In [22]:
from openmdao.api import Problem, ScipyOptimizeDriver, ExecComp, IndepVarComp, ExplicitComponent, Group, NonlinearBlockGS
import numpy as np

In previous tutorials, you built and optimized models comprised of only a single component. Now, we’ll work through a slightly more complex problem that involves two disciplines and hence two main components. You’ll learn how to group components together into a larger model and how to use different kinds of nonlinear solvers to converge multidisciplinary models with coupling between components.

### Sellar - A Simple Two-Discipline Problem
The Sellar problem is a simple two discipline toy problem with each discipline described by a single equation. The output of each component feeds into the input of the other, which creates a coupled model that needs to be converged in order for the outputs to be valid. You can see the coupling between the two disciplines show up through the $y_1$ and $y_2$ variables in the XDSM diagram describing the problem structure below:

<img src="http://openmdao.org/twodocs/versions/2.0.0/_images/sellar_xdsm.png" width="600">

### Building the Disciplinary Components
In the component definitions, there is a call to declare_partials in the setup method that looks like this:
```python
self.declare_partials('*', '*', method='fd')
```
This tells OpenMDAO to approximate all the partial derivatives of that component using finite-difference. The default settings will use forward difference with an absolute step size of 1e-6, but you can change the FD settings to work well for your component.

In [23]:
class SellarDis1(ExplicitComponent):
    

    def setup(self):

        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Local Design Variable
        self.add_input('x', val=0.)

        # Coupling parameter
        self.add_input('y2', val=1.0)

        # Coupling output
        self.add_output('y1', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        
        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        x1 = inputs['x']
        y2 = inputs['y2']

        outputs['y1'] = z1**2 + z2 + x1 - 0.2*y2

In [24]:
class SellarDis2(ExplicitComponent):
    

    def setup(self):
        # Global Design Variable
        self.add_input('z', val=np.zeros(2))

        # Coupling parameter
        self.add_input('y1', val=1.0)

        # Coupling output
        self.add_output('y2', val=1.0)

        # Finite difference all partials.
        self.declare_partials('*', '*', method='fd')

    def compute(self, inputs, outputs):
        

        z1 = inputs['z'][0]
        z2 = inputs['z'][1]
        y1 = inputs['y1']

        # Note: this may cause some issues. However, y1 is constrained to be
        # above 3.16, so lets just let it converge, and the optimizer will
        # throw it out
        if y1.real < 0.0:
            y1 *= -1

        outputs['y2'] = y1**.5 + z1 + z2

### Grouping and Connecting Components
We want to build the model represented by the XDSM diagram above. We’ve defined the two disciplinary components, but there are still the three outputs of the model that need to be computed. Additionally, since we have the computations split up into multiple components, we need to group them all together and tell OpenMDAO how to pass data between them.

In [25]:
class SellarMDA(Group):
    

    def setup(self):
        indeps = self.add_subsystem('indeps', IndepVarComp(), promotes=['*'])
        indeps.add_output('x', 1.0)
        indeps.add_output('z', np.array([5.0, 2.0]))

        cycle = self.add_subsystem('cycle', Group(), promotes=['*'])
        d1 = cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'], promotes_outputs=['y1'])
        d2 = cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'], promotes_outputs=['y2'])

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = NonlinearBlockGS()

        self.add_subsystem('obj_cmp', ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                           z=np.array([0.0, 0.0]), x=0.0),
                           promotes=['x', 'z', 'y1', 'y2', 'obj'])

        self.add_subsystem('con_cmp1', ExecComp('con1 = 3.16 - y1'), promotes=['con1', 'y1'])
        self.add_subsystem('con_cmp2', ExecComp('con2 = y2 - 24.0'), promotes=['con2', 'y2'])

In [53]:
prob = Problem()

prob.model = SellarMDA()

prob.setup()
prob['indeps.x'] = 2.
prob['indeps.z'] = [-1., -1.]

prob.run_model()
print('x=', prob['indeps.x'][0])
print('z=', prob['indeps.z'])
print('y1=', prob['y1'][0])
print('y2=', prob['y2'][0])
print('obj=', prob['obj'][0])
print('con1=', prob['con1'][0])
print('con2=', prob['con2'][0])


=====
cycle
=====
NL: NLBGS Converged in 9 iterations
x= 2.0
z= [-1. -1.]
y1= 2.109516505912702
y2= -0.5475825304298003
obj= 6.838584497976375
con1= 1.050483494087298
con2= -24.5475825304298


We’re working with a new type of class: Group. This is the container that lets you build up complex model hierarchies. Groups can contain other groups, components, or combinations of groups and components.

You can directly create instances of Group to work with, or you can sub-class from it to define your own custom groups. We’re doing both things here. First, we define our own custom Group sub-class called SellarMDA. In our run-script well create an instance of SellarMDA to actually run it. Then inside the setup method of SellarMDA we’re also working directly with a group instance by doing this:

```python
cycle = self.add_subsystem('cycle', Group(), promotes=['*'])
d1 = cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z', 'y2'], promotes_outputs=['y1'])
d2 = cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z', 'y1'], promotes_outputs=['y2'])

# Nonlinear Block Gauss Seidel is a gradient-free solver
cycle.nonlinear_solver = NonlinearBlockGS()
```

Our SellarMDA Group, when instantiated, will have a three level hierarchy with itself as the top most level.

<img src="http://openmdao.org/twodocs/versions/2.0.0/_images/sellar_tree.png" width="600">

### Why do we create the cycle sub-group?
There is a circular data dependence between d1 and d2 that needs to be converged with a nonlinear solver in order to get a valid answer. Its a bit more efficient to put these two components into their own sub-group, so that we can iteratively converge them by themselves, before moving on to the rest of the calculations in the model. Models with cycles in them are often referred to as Multidisciplinary Analyses or MDA for short. You can pick which kind of solver you would like to use to converge the MDA. The most common choices are:

* `NonlinearBlockGaussSeidel`
* `NewtonSolver`

The `NonlinearBlockGaussSeidel` solver, also sometimes called a “fixed-point iteration solver”, is a gradient-free method that works well in many situations. More tightly coupled problems, or problems with instances of `ImplicitComponent` that don’t implement their own `solve_nonlinear` method, will require the `NewtonSolver`.

The sub-group, named cycle, is useful here, because it contains the multidisciplinary coupling of the Sellar problem. This allows us to assign the non-linear solver to cycle to just converge those two components, before moving on to the final calculations for the obj_cmp, con_cmp1, and con_cmp2 to compute the actual outputs of the problem.

### Promoting variables with the same name connects them
The data connections in this model are made via promotion. OpenMDAO will look at each level of the hierarchy and find all the connect all output-input pairs with the same name.

### ExecComp is a helper component for quickly defining components for simple equations
A lot of times in your models, you need to define a new variable as a simple function of other variables. OpenMDAO provides a helper component to make this easier, called ExecComp. Its fairly flexible, allowing you to work with scalars or arrays, work with units, and call basic math funcsion (e.g. sin or exp).

## Linking Variables with Promotion vs Connection
In the previous tutorial we built up a model of the Sellar problem using two disciplinary components and a few ExecComps. In order to get OpenMDAO to pass the data between all the components, we linked everything up using promoted variables so that data passed from outputs to inputs with the same promoted name.
Promoting variables is often a convenient way to establish the data passing links from outputs to inputs. However, you can also use calls to the connect method in order to link outputs to inputs without having to promote anything. Here is how you would define the same Sellar model using:
1. Variable promotion
2. Connect statements
3. Both variable promotion and connect statements

All three will give the exact same answer, but the way you address the variables will be slightly different in each one.

In [30]:
class SellarMDAConnect(Group):
    

    def setup(self):
        indeps = self.add_subsystem('indeps', IndepVarComp())
        indeps.add_output('x', 1.0)
        indeps.add_output('z', np.array([5.0, 2.0]))

        cycle = self.add_subsystem('cycle', Group())
        d1 = cycle.add_subsystem('d1', SellarDis1())
        d2 = cycle.add_subsystem('d2', SellarDis2())
        cycle.connect('d1.y1', 'd2.y1')
        cycle.connect('d2.y2', 'd1.y2')

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = NonlinearBlockGS()

        self.add_subsystem('obj_cmp', ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                               z=np.array([0.0, 0.0]), x=0.0))

        self.add_subsystem('con_cmp1', ExecComp('con1 = 3.16 - y1'))
        self.add_subsystem('con_cmp2', ExecComp('con2 = y2 - 24.0'))

        self.connect('indeps.x', ['cycle.d1.x', 'obj_cmp.x'])
        self.connect('indeps.z', ['cycle.d1.z', 'cycle.d2.z', 'obj_cmp.z'])
        self.connect('cycle.d1.y1', ['obj_cmp.y1', 'con_cmp1.y1'])
        self.connect('cycle.d2.y2', ['obj_cmp.y2', 'con_cmp2.y2'])

The exact same model results can be achieved using connect statements instead of promotions. However, take careful note of how the variables are addressed in those connect and print statements.

In [54]:
prob = Problem()

prob.model = SellarMDAConnect()

prob.setup()
prob['indeps.x'] = 2.
prob['indeps.z'] = [-1., -1.]

prob.run_model()
print('x=', prob['indeps.x'][0])
print('z=', prob['indeps.z'])
print('y1=', prob['cycle.d1.y1'][0])
print('y2=', prob['cycle.d2.y2'][0])
print('obj=', prob['obj_cmp.obj'][0])
print('con1=', prob['con_cmp1.con1'][0])
print('con2=', prob['con_cmp2.con2'][0])


=====
cycle
=====
NL: NLBGS Converged in 9 iterations
x= 2.0
z= [-1. -1.]
y1= 2.109516505912702
y2= -0.5475825304298003
obj= 6.838584497976375
con1= 1.050483494087298
con2= -24.5475825304298


# Optimizing the Sellar Problem

In the previous section of this tutorial we showed you how to define the Sellar model and run it directly. Now let's see how we can optimize the Sellar problem to minimize the objective function. Here is the mathematical problem formulation for the Sellar optimiziation problem:

\begin{align}
\text{min}: & \ \ \ & x_1^2 + z_2 + y_1 + e^{-y_2} \\
\text{w.r.t.}: & \ \ \ &  x_1, z_1, z_2 \\
\text{subject to}: & \ \ \ & \\
& \ \ \ & 3.16 - y_1 <=0 \\
& \ \ \ & y_2 - 24.0 <=0
\end{align}

In [55]:
prob = Problem()
prob.model = SellarMDA()


prob.driver = ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)


prob.setup()
prob.set_solver_print(level=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()


prob.run_driver()

print('minimum found at:')
print('x=', prob['indeps.x'][0])
print('z=', prob['indeps.z'])
print('y1=', prob['y1'][0])
print('y2=', prob['y2'][0])
print('obj=', prob['obj'][0])
print('con1=', prob['con1'][0])
print('con2=', prob['con2'][0])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3.1833939517831795
            Iterations: 6
            Function evaluations: 6
            Gradient evaluations: 6
Optimization Complete
-----------------------------------
minimum found at:
x= 1.7492161945256356e-14
z= [1.97763888 0.        ]
y1= 3.16000000014471
y2= 3.7552777670179114
obj= 3.1833939517831795
con1= -1.447100217433217e-10
con2= -20.244722232982088


### Approximate the total derivatives with finite difference
In this case we’re using the SLSQP algorithm, which is a gradient based optimization approach. Up to this point none of our components have provided any analytic derivatives, so we’ll just finite-difference across the whole model to approximate the derivatives. That is accompished by this line of code:
```
 prob.model.approx_total_derivs()
 ```
